In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('gmr520grainsSD.xlsx')
df = df.fillna('null')
np_df = df.to_numpy()

In [3]:
COL_SEASON = 1
COL_COUNTRY = 1
COL_VL_VALUE = 4
COL_ESTIMATE_TYPE = 2

COL_OPENING = 3
COL_COMMODITY = 1
END_TABEL_SYBOL = 1

In [4]:
def count_table_in_file(np_df, COL_OPENING = COL_OPENING):
    return len([row for row in np_df if row[COL_OPENING] == 'Opening'])


def list_start_table(np_df, COL_OPENING = COL_OPENING):
    list_start = []
    for i, row in enumerate(np_df):
        if row[COL_OPENING] == 'Opening':
            list_start.append(i-1)
    return list_start


def get_kind_table(np_df, COL_COMMODITY = COL_COMMODITY):
    list_start = list_start_table(np_df)
    list_kinds_table = []
    for start in list_start:
        for row in np_df[start:]:
            if row[COL_COMMODITY] != 'null':
                list_kinds_table.append(row[COL_COMMODITY])
                break
    return list_kinds_table


def get_element_unit(np_df):
    return [clear_from_null_np_array(np_df[start]) for start in list_start_table(np_df)]
        

def clear_from_null_np_array(np_array):
    return [x for x in np_array if x != 'null'][0]


def find_end_tabels(np_df, END_TABEL_SYBOL=END_TABEL_SYBOL):
    list_start = list_start_table(np_df)
    for row in np_df[list_start[-1]:]:
        if row[END_TABEL_SYBOL].split()[0] == '#':
            itemindex = np.where(np_df==row[END_TABEL_SYBOL])
    return itemindex[0][0]

<b>Example:</b> <br/>
country_region - Argentina<br />
commodity - Wheat<br />
element - Production<br />
season - 2020/21<br />
vl_value - 17,6<br />
element_unit - Million tonnes<br />
valid_from - 10.11.2020<br />
valid_to - 01.01.9999<br />
estimate type - f'cast<br />
marketing_period - Dec/Nov<br />

In [13]:
list_country_region = []
list_commodity = []
list_element = [] #? 
list_season = []
list_vl_value = []
list_element_unit = []
list_estimate_type = []
list_marketing_period = []

list_start = list_start_table(np_df)
list_kind_table = get_kind_table(np_df)
list_element = get_element_unit(np_df)

for i, start in enumerate(list_start):
    if i < len(list_start) - 1:
        end = list_start[i+1]
    else:
        end = find_end_tabels(np_df)
    cntr = False
    for row in np_df[start:end]:
        if (row[COL_COUNTRY] != 'null' and 
            row[COL_COUNTRY] not in list_kind_table and 
            not row[COL_COUNTRY][0].isnumeric()):
            cntr = row[COL_COUNTRY]
        elif row[COL_COUNTRY] == 'null':
            cntr = False
        if cntr and row[COL_COUNTRY][0].isnumeric():
            list_cntr = cntr.split('(')
            if len(list_cntr) == 1:
                marketing_period = ''
            else:
                marketing_period = list_cntr[1][:-1]
                
            list_country_region.append(list_cntr[0].strip())
            list_commodity.append(list_kind_table[i])
            list_season.append(row[COL_SEASON])
            list_vl_value.append(row[COL_VL_VALUE])
            list_element_unit.append(list_element[i])
            list_estimate_type.append(row[COL_ESTIMATE_TYPE])
            list_marketing_period.append(marketing_period)

dict_excel = {'country_region':list_country_region
             ,'commodity':list_commodity
             ,'season':list_season
             ,'vl_value':list_vl_value
             ,'element_unit':list_element_unit
             ,'estimate_type':list_estimate_type
             ,'marketing_period':list_marketing_period}

new_df = pd.DataFrame(dict_excel)

In [15]:
new_df.head(100)

,country_region,commodity,season,vl_value,element_unit,estimate_type,marketing_period
0,Argentina **,TOTAL GRAINS,2019/20,84.67667,Million tons,est.,
1,Argentina **,TOTAL GRAINS,2020/21,80.49500,Million tons,f'cast,
2,Argentina **,TOTAL GRAINS,2021/22,86.96700,Million tons,proj.,
3,Australia **,TOTAL GRAINS,2019/20,25.66357,Million tons,est.,
4,Australia **,TOTAL GRAINS,2020/21,50.18900,Million tons,f'cast,
...,...,...,...,...,...,...,...
95,South Africa,MAIZE,2022/23,15.30000,Million tons,proj.,May/Apr
96,Ukraine,MAIZE,2019/20,35.88700,Million tons,est.,Oct/Sep
97,Ukraine,MAIZE,2020/21,30.00000,Million tons,f'cast,Oct/Sep
98,Ukraine,MAIZE,2021/22,37.34500,Million tons,proj.,Oct/Sep


In [16]:
new_df.to_csv('test.csv', index=False, sep=';');